# Object-Oriented Programming

Upon reading the chapter title, one could wonder why object-oriented programming
(OOP) is introduced only now.  We have used objects
since Chapter 2, and we started making our own
classes and object types
in Chapter 8, so what is new in Chapter 9? The answer is that
the term OOP can have two different meanings.
The first simply involves programming with objects and classes, which
we introduced in Chapter 8, and is more commonly
referred to as object-*based* programming. The second meaning
of OOP is programming with *class hierarchies*, which are
families of classes that inherit their methods and attributes from each other.
This is the topic of the present chapter. We will learn how to collect
classes in families (hierarchies) and let child classes inherit
attributes and methods from parent classes.

# Class Hierarchies and Inheritance
A class hierarchy is a family of closely related classes organized in a
hierarchical manner. A key concept is *inheritance*, which means that
child classes can inherit attributes and methods from parent classes.
A typical strategy is to write a general class as a base class (or parent class)
and then let special cases be represented as subclasses (child classes).
This approach can often save much typing and code duplication. As usual, we
introduce the topic by looking at some examples.

### Classes for lines and parabolas.

As a first example, let us create a class for representing and evaluating straight lines,
$y=c_0 + c_1x$. Following the concepts and ideas introduced in
Chapter 8, the implementation of the class can look like

In [1]:
import numpy as np

class Line:
    def __init__(self, c0, c1):
        self.c0, self.c1 = c0, c1

    def __call__(self, x):
        return self.c0 + self.c1*x

    def table(self, L, R, n):
        """Return a table with n points for L <= x <= R."""
        s = ''
        for x in np.linspace(L, R, n):
            y = self(x)
            s += f'{x:12g} {y:12g}\n'
        return s

We see that we have equipped the class with a standard constructor, a `__call__` special method
for evaluating the linear function, and a method `table` for writing a table of $x$ and $y$ values.
Say we now want to write a similar class for evaluating a parabola $y=c_0 + c_1x + c_2x^2$.
The code could look like

In [2]:
class Parabola:
    def __init__(self, c0, c1, c2):
        self.c0, self.c1, self.c2 = c0, c1, c2

    def __call__(self, x):
        return self.c2*x**2 + self.c1*x + self.c0

    def table(self, L, R, n):
        """Return a table with n points for L <= x <= R."""
        s = ''
        for x in linspace(L, R, n):
            y = self(x)
            s += f'{x:12g} {y:12g}\n'
        return s

We observe that the two classes are nearly identical, differing only in the parts
that involve `c2`. Although we could very quickly just copy all the code from the `Line` class
and edit the small parts that are needed, such duplication of code is usually a bad idea.
At some point, we may need change the code, for instance, to correct an error or
improve the functionality, and having to make the same change in multiple
places often leads to time-consuming errors. So, is there a way we can utilize
the class `Line` code in `Parabola` without resorting to copying and pasting?
This is exactly what inheritance is about.

To introduce inheritance, let us first look at the following class definition:

In [3]:
class Parabola(Line):
    pass

Here `pass` is just a Python keyword that can be used wherever Python expects
to find code, but we do not want to define anything. So, at
first sight, this `Parabola` class seems to be empty, but notice the class
definition `class Parabola(Line)`, which means that `Parabola` is a subclass of
`Line` and *inherits* all its methods and attributes. The new `Parabola` class therefore has
attributes `c0` and `c1` and three methods `__init__`, `__call__`, and
`table`. `Line` is a *base class* (or parent class, superclass) , and `Parabola` is
a *subclass*  (or child class, derived class). The new `Parabola` class, therefore,
is not as useless as it first seemed, but it is still just a copy of the `Line`
class. To make the class represent a parabola, we need to add the missing code,
that is, the code that differs between `Line` and `Parabola`. When creating such
subclasses, the principle is to reuse as much as possible from the base class,
only add what is needed in the subclass, and avoid duplicating
code. Inspecting the two original classes above, we see that the
`Parabola` class must add code to `Line`'s constructor (an extra `c2` attribute)
and an extra term in `__call__`, but `table` can be used unaltered.
The full definition of `Parabola` as a subclass of `Line` becomes the following:

In [4]:
class Parabola(Line):
    def __init__(self, c0, c1, c2):
        super().__init__(c0, c1)  # Line stores c0, c1
        self.c2 = c2

    def __call__(self, x):
        return super().__call__(x) + self.c2*x**2

To maximize code reuse, we allow the `Parabola` class to call the methods from `Line`,
and then add the missing parts.  A subclass can always access
its base class bt using the built-in function `super()`, and this is the preferred way
to call methods from the base class. We could, however, also
use the class name directly, for instance `Line.__init__(self,c0,c1)`.
Generally, these two methods for invoking superclass methods look like
the following:

```Python
        SuperClassName.method(self, arg1, arg2, ...)
        super().method(arg1, arg2, ...)
```

Notice the difference between the two approaches. When using the class
name directly, we need to include `self` as the first argument, whereas this aspect
is handled automatically when using `super()`. The use of `super()` is usually
preferred, but in most cases the two approaches are equivalent.

To summarize this first example, the main benefits of
introducing the subclass are as follows:
  * Class `Parabola` just adds code to the already existing code in class `Line`, with no duplication of the code for storing `c0` and `c1` and computing $c_0+c_1x.$

  * Class `Parabola` also has a `table` method; it is inherited and does not need to be written.

  * `__init__` and `__call__` are *overridden* or *redefined* in the subclass, with no code duplication.

We can use the `Parabola` class and call its methods just as if they were implemented in the class directly:

In [5]:
p = Parabola(1, -2, 2)
p1 = p(2.5)
print(p1)
print(p.table(0, 1, 3))

### The real meaning of inheritance.

From a practical viewpoint, and for the examples in this book, the point of
inheritance is to reuse methods and attributes from the
base class and minimize code duplication. On a more theoretical level, inheritance
should be thought of as an "is-a" relationship between the
the two classes. By this we mean that if `Parabola` is a subclass of `Line`,
an instance of `Parabola` is also a `Line` instance. The `Parabola` class
is thought of as a special case of the `Line` class, and therefore
every `Parabola` is also a `Line`, but not vice versa. We can
check class type and class relations with the built-in
functions `isinstance(obj, type)` and `issubclass(subclassname, superclassname)`:

In [6]:
l = Line(-1, 1)
isinstance(l, Line)
isinstance(l, Parabola)
p = Parabola(-1, 0, 10)
isinstance(p, Parabola)

In [7]:
isinstance(p, Line)

In [8]:
issubclass(Parabola, Line)

In [9]:
issubclass(Line, Parabola)

In [10]:
p.__class__ == Parabola

In [11]:
p.__class__.__name__   # string version of the class name

We will not use these methods much in practical applications, but
they are very useful for gaining a feel for class relationships when
learning OOP.


Mathematically oriented readers might have noticed a logical fault in the small
class hierarchy we have presented so far. We stated that a subclass
is usually thought of as a special case of the base class, but a parabola is not
really a special case of a straight line. It is the
other way around, since a line $c_0+c_1x$ is a parabola $c_0+c_1x+c_2x^2$ with $c_2=0$.
Could then `Line`, then, be a subclass of `Parabola`? Certainly, and many
will prefer this relation between a line and a parabola, since it follows the
usual is-a relationship between a subclass and its base.
The code can look like:

In [12]:
class Parabola:
    def __init__(self, c0, c1, c2):
        self.c0, self.c1, self.c2 = c0, c1, c2

    def __call__(self, x):
        return self.c2*x**2 + self.c1*x + self.c0

    def table(self, L, R, n):
        """Return a table with n points for L <= x <= R."""
        s = ''
        for x in linspace(L, R, n):
            y = self(x)
            s += '%12g %12g\n' % (x, y)
        return s

class Line(Parabola):
    def __init__(self, c0, c1):
        super().__init__(c0, c1, 0)

Notice that this version allows even more code reuse than the previous one,
since both `__call__` and `table` can be reused without changes.


# Example: Classes for Numerical Differentiation
Common tasks in scientific computing, such as differentiation and integration,
can be carried out with a large variety of numerical methods. Many such methods
are closely related, and can be easily grouped into families of methods that
are very suitable for implementation in a class hierarchy.
As a first example, we consider methods for numerical differentiation. The
simplest formula is a one-sided finite difference:

$$
f'(x) \approx {f(x+h)-f(x)\over h} ,
$$

which can be implemented in the following class:

In [13]:
class Derivative:
    def __init__(self, f, h=1E-5):
        self.f = f
        self.h = float(h)

    def __call__(self, x):
        f, h = self.f, self.h      # make short forms
        return (f(x+h) - f(x))/h

To use the `Derivative` class, we simply define a function `f(x)`,
create an instance of the class, and call it as if it were a regular function
(effectively calling the `__call__` method behind the scenes):

In [14]:
from math import exp, sin, pi

def f(x):
    return exp(-x)*sin(4*pi*x)

dfdx = Derivative(f)
print(dfdx(1.2))

However, numerous other formulas can be used for numerical differentiation,
for instance

$$
\begin{align*}
f'(x) &= \frac{f(x+h)-f(x)}{h} + {\cal O}(h) ,\\
f'(x) &= \frac{f(x)-f(x-h)}{h} + {\cal O}(h) ,\\
f'(x) &= \frac{f(x+h)-f(x-h)}{2h} + {\cal O}(h^2) ,\\
f'(x) &=  \frac{4}{3}\frac{f(x+h)-f(x-h)}{2h}
  -\frac{1}{3}\frac{f(x+2h) - f(x-2h)}{4h} + {\cal O}(h^4), \\
f'(x) &=  \frac{3}{2}\frac{f(x+h)-f(x-h)}{2h}
  -\frac{3}{5}\frac{f(x+2h) - f(x-2h)}{4h} + \nonumber\\
&  \frac{1}{10}\frac{f(x+3h) - f(x-3h)}{6h} + {\cal O}(h^6), \\
f'(x) &=  \frac{1}{h}\left(
-\frac{1}{6}f(x+2h) + f(x+h) - \frac{1}{2}f(x) - \frac{1}{3}f(x-h)\right)
  + {\cal O}(h^3).
\end{align*}
$$

We can easily make a module that offers multiple formulas, as follows:

In [15]:
class Forward1:
    def __init__(self, f, h=1E-5):
        self.f, self.h = f, h

    def __call__(self, x):
        f, h = self.f, self.h
        return (f(x+h) - f(x))/h

class Central2:
    def __init__(self, f, h=1E-5):
        self.f, self.h = f, h

    def __call__(self, x):
        f, h = self.f, self.h
        return (f(x+h) - f(x-h))/(2*h)

class Central4:
    def __init__(self, f, h=1E-5):
        self.f, self.h = f, h

    def __call__(self, x):
        f, h = self.f, self.h
        return (4./3)*(f(x+h)   - f(x-h))  /(2*h) - \
                (1./3)*(f(x+2*h) - f(x-2*h))/(4*h)

The problem with this code is, of course, that all the constructors are
identical, so we duplicate a great deal of code. Although the duplication of
this simple constructor might not be a big problem, it can easily lead to
errors if we want to change the constructor later, and it is therefore worth avoiding.
As mentioned above, a general idea of OOP is to place
code common to many classes in a superclass and to have that code be inherited
by the subclasses.
In this case, we can make a superclass containing the constructor and
let the different subclasses implement their own version of the `__call__` method.
The superclass will be very simple and not really useful on its own:

In [16]:
class Diff:
    def __init__(self, f, h=1E-5):
        self.f, self.h = f, h

The subclasses for the first-order forward formula and the second-
and fourth-order central difference formulas can then look like

In [17]:
class Forward1(Diff):
    def __call__(self, x):
        f, h = self.f, self.h
        return (f(x+h) - f(x))/h

class Central2(Diff):
    def __call__(self,x):
        f, h = self.f, self.h
        return (f(x+h)-f(x-h))/(2*h)

class Central4(Diff):
    def __call__(self, x):
        f, h = self.f, self.h
        return (4./3)*(f(x+h)   - f(x-h))  /(2*h) - \
               (1./3)*(f(x+2*h) - f(x-2*h))/(4*h)

To use this simple class hierarchy in an example, say, we want to compute
the derivative of $f(x)=\sin x$ for $x=\pi$ with the fourth-order
central difference formula:

In [18]:
from math import sin, pi
mycos = Central4(sin)
mycos(pi)

Here, the line `mycos = Central4(sin)` creates an instance of the
`Central4` class by calling the constructor inherited from the superclass,
while `mycos(pi)` calls the `__call__` method implemented in the subclass.

As indicated by the $O(h^n)$ terms in the formulas above, the methods
have different levels of accuracy. We can empirically investigate the
accuracy of the numerical differentiation formulas, using the class hierarchy
created above. Using $f(x)=\sin x, x = \pi/4$ as an example, the code can look like

In [19]:
from math import pi, sin, cos
import numpy as np

h = [1.0/(2**i) for i in range(5)]
ref = cos(pi/4)

print(f'     h      Forward1      Central2      Central4')
for h_ in h:
    f1 = Forward1(sin,h_); c2 = Central2(sin,h_); c4 = Central4(sin,h_)
    e1 = abs(f1(pi/4)-ref)
    e2 = abs(c2(pi/4)-ref)
    e4 = abs(c4(pi/4)-ref)
    print(f'{h_:1.8f}  {e1:1.10f}  {e2:>1.10f}  {e4:>1.10f}')

Notice that we create new instances `f1, c2`, and `c4` for each iteration of the
loop, since we want a new value of $h$ in the formula. A more elegant solution
could be to add a new method named `set_stepsize(h)` or similar, that would
allow us to adjust $h$ for an existing instance. Such a method could easily be
added to the superclass and inherited by all subclasses. An examination of
the output numbers confirm that the three methods behave as expected. For
each row, we reduce $h$ by a factor of two, and the errors are reduced by a factor
of about two, four, and 16, respectively. This result is consistent with the
theoretical accuracy of the
formulas, which states that the errors should be proportional to $h$, $h^2$, and $h^4$, respectively.

# Example: Classes for Numerical Integration
Just as numerical differentiation, numerical integration is a mainstay of
computational mathematics. There are numerous methods to choose from, and they
can all be written on the form

$$
\int_a^b f(x)dx \approx \sum_{i=0}^{n-1} w_i f(x_i) .
$$

and the
Based on this general formula, different methods are realized by choosing the integration points
$x_i$ and associated weights $w_i$. For instance, the trapezoidal rule has

$$
x_i = a+ih, \quad w_0=w_{n-1}={h\over2},\ w_i=h\ (i\neq 0,n-1) ,
$$

with  $h=(b-a)/(n-1)$, the midpoint rule has

$$
x_i = a + {h\over 2} + ih,\quad w_i=h ,
$$

with $h=(b-a)/n$, and Simpson's rule has

$$
\begin{align*}
x_i &= a+ih,\quad h={b-a\over n-1}, \\
w_0 &=w_{n-1}={h\over6}, \\
w_i &= {h\over3}\hbox{ for }i\hbox{ even},\quad w_i={2h\over3}\hbox{ for }i\hbox{ odd} .
\end{align*}
$$

Other methods have more complicated formulas for $w_i$ and $x_i$, and some methods
choose the points randomly (e.g., Monte Carlo integration).

A numerical integration formula can be implemented as a class, with
$a$, $b$, and $n$ as attributes and an `integrate` method to evaluate the formula and compute
the integral. As with the family of numerical differentiation methods considered above,
all such classes will be quite similar. The evaluation of $\sum_jw_jf(x_j)$ is the same,
and the only difference between the methods is the definition of the points and
weights. Following the ideas above, it makes sense to place all common code in
a superclass, and code specific to the different methods in subclasses. Here, we can
put $\sum_jw_jf(x_j)$ in a superclass (method `integrate`), and let the subclasses extend this
class with code specific to a specific formula, that is, the choices of
$w_i$ and $x_i$. This method-specific code
can be placed inside a method, for instance, named `construct_rule`. The superclass for the numerical
integration hierarchy can look like

In [20]:
class Integrator:
    def __init__(self, a, b, n):
        self.a, self.b, self.n = a, b, n
        self.points, self.weights = self.construct_method()

    def construct_method(self):
        raise NotImplementedError('no rule in class %s' % \
                                  self.__class__.__name__)

    def integrate(self, f):
        s = 0
        for i in range(len(self.weights)):
            s += self.weights[i]*f(self.points[i])
        return s

    def vectorized_integrate(self, f):
        # f must be vectorized for this to work
        return dot(self.weights, f(self.points))

Notice the implementation of `construct_method`, which will raise an error if
it is called, indicating that the only purpose of `Integrator` is as a
superclass, and it should not be used directly. Alternatively, we could, of course,
just not include the `construct_method` method in the superclass at all.
However, the approach used here makes it even more obvious that the class is
just a superclass and that this method needs to be implemented in subclasses.

The superclass provides a common framework for implementing the different methods, which can
then be realized as subclasses. The trapezoidal and midpoint methods can
be implemented as follows:

In [21]:
class Trapezoidal(Integrator):
    def construct_method(self):
        h = (self.b - self.a)/float(self.n - 1)
        x = linspace(self.a, self.b, self.n)
        w = zeros(len(x))
        w[1:-1] += h
        w[0] = h/2;  w[-1] = h/2
        return x, w

class Midpoint(Integrator):
    def construct_method(self):
        a, b, n = self.a, self.b, self.n  # quick forms
        h = (b-a)/float(n)
        x = np.linspace(a + 0.5*h, b - 0.5*h, n)
        w = np.zeros(len(x)) + h
	return x, w

The more complex Simpson's rule can be added in the following subclass:

In [22]:
class Simpson(Integrator):
    def construct_method(self):
        if self.n % 2 != 1:
            print ’n=%d must be odd, 1 is added’ % self.n
            self.n += 1
        x = np.linspace(self.a, self.b, self.n)
        h = (self.b - self.a)/float(self.n - 1)*2
        w = np.zeros(len(x))
        w[0:self.n:2] = h*1.0/3
        w[1:self.n-1:2] = h*2.0/3
        w[0] /= 2
        w[-1] /= 2
        return x, w

Simpson's rule is more complex because it uses different weights for odd and even points. We
present all the details here for completeness, but it is not really
necessary to study the details of all the formulas. The important parts
here are the class design and usage of the class hierarchy.

To demonstrate how the class can be used, let us compute the integral
$\int_0^2 x^2dx$ using 101 points:

In [23]:
def f(x):
    return x*x

simpson = Simpson(0, 2, 101)
print(simpson.integrate(f))
trapez = Trapezoidal(0,2,101)
print(trapez.integrate(f))

The program flow in this case might not be entirely obvious. When we construct
the instance with `method = Simpson(0, 2, 101)`, the superclass constructor is invoked,
but this method then calls `construct_method` in class `Simpson`. The
call `method.integrate(f)` then invokes the `integrate` method inherited
from the superclass. However, as users of
the class, none of these details really matter to us. We use the `Simpson` class just as if all
the methods were implemented directly in the class, regardless of whether they are actually
inherited from another class.